In [ ]:
!pip install -U ultralytics wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 30.7 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.18.7
    Uninstalling wandb-0.18.7:
      Successfully uninstalled wandb-0.18.7


In [ ]:
import random
import shutil
import torch
import yaml
import os

import numpy as np

from pathlib import Path

from ultralytics import YOLO

from sklearn.model_selection import train_test_split

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
shutil.copy2('/content/drive/MyDrive/ITaS/data/best.pt', 'best.pt')

'best.pt'

In [ ]:
import wandb
from wandb.integration.ultralytics import add_wandb_callback

wandb.login(key='1de7addec37341330a8aef9bfc7be382cc9c2824')

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sunests. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
def create_yaml(output_yaml_path, train_image_dir, val_image_dir, test_image_dir=' ', nc=10):

    names = ['lungs', 'trachea', 'bronchitis', 'pneumonia/bronchopneumonia', 'pulmonary edema',
             'hydrothorax', 'pneumothorax', 'tracheal collapse', 'neoplasm', 'atelectasis']

    yaml_data = {
        'names': names,
        'nc': nc,
        'train': train_image_dir,
        'val': val_image_dir,
        'test': test_image_dir
    }

    with open(output_yaml_path, 'w') as j:
        yaml.dump(yaml_data, j, default_flow_style=False)

In [ ]:
def calculation_least_confidence_dataset(
    confidence_threshold: float = 0.1
):
    path_to_dataset = '/content/drive/MyDrive/ITaS/data/full_train'

    # train_labels_paths не создаем, так как пройдя через алгоритм Least Confidence
    # мы получим урезанный список картинок, которму train_labels_paths не будет соответствовать.
    # Легче потом просто использовать str.replace(), так как наименование файлов одинаковое.
    train_images_paths = []

    val_images_paths = []
    val_labels_paths = []

    # Проходимся по директории полного датасета
    for root, _, files in os.walk(path_to_dataset):
        for file in files:
            # train часть
            if r'/train/images' in root:
                train_images_paths.append(os.path.join(root, file))
            # val часть
            elif r'/val/images' in root:
                val_images_paths.append(os.path.join(root, file))
            elif r'/val/labels' in root:
                val_labels_paths.append(os.path.join(root, file))

    # Алгоритм Least Confidence
    model = YOLO('best.pt')

    selected_train_images_samples = []
    for i, image_path in enumerate(train_images_paths):
        print(f'{i}/{len(train_images_paths)} ->> {image_path}')
        results = model.predict(image_path, conf=confidence_threshold)
        result = results[0]

        selected_train_images_samples.append({
            'image_path': image_path,
            'min_confidence': min(result.boxes.conf)
        })

    # Сортируем по приоритету
    selected_train_images_samples.sort(key=lambda x: x['min_confidence'])

    # Вычленяем только пути у сортированного словаря
    selected_train_images_samples = [image['image_path'] for image in selected_train_images_samples]
    selected_train_labels_samples = [path.replace('images', 'labels').replace('.jpg', '.txt') for path in selected_train_images_samples]

    return (selected_train_images_samples, selected_train_labels_samples, val_images_paths, val_labels_paths)

In [ ]:
least_confidence_full_dataset = calculation_least_confidence_dataset()
least_confidence_full_dataset

0/1105 ->> /content/drive/MyDrive/ITaS/data/full_train/train/images/1.1.3.1.40.jpg

image 1/1 /content/drive/MyDrive/ITaS/data/full_train/train/images/1.1.3.1.40.jpg: 640x928 1 lungs, 1 trachea, 7968.2ms
Speed: 17.0ms preprocess, 7968.2ms inference, 54.3ms postprocess per image at shape (1, 3, 640, 928)
1/1105 ->> /content/drive/MyDrive/ITaS/data/full_train/train/images/1.2.1.1.167.jpg

image 1/1 /content/drive/MyDrive/ITaS/data/full_train/train/images/1.2.1.1.167.jpg: 640x928 1 lungs, 1 trachea, 1 pneumonia/bronchopneumonia, 8603.8ms
Speed: 10.5ms preprocess, 8603.8ms inference, 27.0ms postprocess per image at shape (1, 3, 640, 928)
2/1105 ->> /content/drive/MyDrive/ITaS/data/full_train/train/images/1.1.1.1.202.jpg

image 1/1 /content/drive/MyDrive/ITaS/data/full_train/train/images/1.1.1.1.202.jpg: 576x928 1 lungs, 1 trachea, 8674.1ms
Speed: 5.6ms preprocess, 8674.1ms inference, 18.1ms postprocess per image at shape (1, 3, 576, 928)
3/1105 ->> /content/drive/MyDrive/ITaS/data/full_tra

KeyboardInterrupt: 

In [ ]:
def create_least_confidence_dataset(
    train_images_path: list,
    train_labels_path: list,
    val_images_path: list,
    val_labels_path: list,
    experiment_name: str,
    percentage_from_top: int = None,
    alg_name: str = 'least_condidence'
):
    train_images_path = train_images_path[:int(len(train_images_path) * (percentage_from_top / 100))]
    train_labels_path = train_labels_path[:int(len(train_labels_path) * (percentage_from_top / 100))]

    train_images_dir = os.path.join('data', alg_name, experiment_name, 'train', 'images')
    val_images_dir = os.path.join('data', alg_name, experiment_name, 'val', 'images')
    train_labels_dir = os.path.join('data', alg_name, experiment_name, 'train', 'labels')
    val_labels_dir = os.path.join('data', alg_name, experiment_name, 'val', 'labels')

    os.makedirs(train_images_dir, exist_ok=True)
    os.makedirs(val_images_dir, exist_ok=True)
    os.makedirs(train_labels_dir, exist_ok=True)
    os.makedirs(val_labels_dir, exist_ok=True)

    for train_image_path, train_label_path in zip(train_images_path, train_labels_path):
        shutil.copy2(train_image_path,
                      os.path.join(
                          'data', alg_name, experiment_name, 'train', 'images',
                          train_image_path[train_image_path.find('images') + 7:]))

        shutil.copy2(train_label_path,
                      os.path.join(
                          'data', alg_name, experiment_name, 'train', 'labels',
                          train_label_path[train_label_path.find('labels') + 7:]))

    for val_image_path, val_label_path in zip(val_images_path, val_labels_path):

        shutil.copy2(val_image_path,
                      os.path.join(
                          'data', alg_name, experiment_name, 'val', 'images',
                          val_image_path[val_image_path.find('images') + 7:]))

        shutil.copy2(val_label_path,
                      os.path.join(
                          'data', alg_name, experiment_name, 'val', 'labels',
                          val_label_path[val_label_path.find('labels') + 7:]))

    yaml_path = os.path.join('data', alg_name, experiment_name, 'data.yaml')
    train_path = os.path.join('train', 'images')
    val_path = os.path.join('val', 'images')
    create_yaml(yaml_path, train_path, val_path)

In [ ]:
create_least_confidence_dataset(
    train_images_path=least_confidence_full_dataset[0],     # train_images
    train_labels_path=least_confidence_full_dataset[1],     # train_labels
    val_images_path=least_confidence_full_dataset[2],       # val_images
    val_labels_path=least_confidence_full_dataset[3],       # val_samples
    percentage_from_top=1,
    experiment_name='1_train'
)

In [ ]:
create_least_confidence_dataset(
    train_images_path=least_confidence_full_dataset[0],     # train_images
    train_labels_path=least_confidence_full_dataset[1],     # train_labels
    val_images_path=least_confidence_full_dataset[2],       # val_images
    val_labels_path=least_confidence_full_dataset[3],       # val_samples
    percentage_from_top=10,
    experiment_name='10_train'
)

In [ ]:
create_least_confidence_dataset(
    train_images_path=least_confidence_full_dataset[0],     # train_images
    train_labels_path=least_confidence_full_dataset[1],     # train_labels
    val_images_path=least_confidence_full_dataset[2],       # val_images
    val_labels_path=least_confidence_full_dataset[3],       # val_samples
    percentage_from_top=20,
    experiment_name='20_train'
)

In [ ]:
def train_yolo_model(main_experiment: str, name: str, seed: int):
    wandb.init(project='ITaS', job_type='training')

    model = YOLO("yolov8m-seg.pt")

    results = model.train(
        data = os.path.join('/content/drive/MyDrive/ITaS' ,'data', main_experiment, name, 'data.yaml'),
        project = 'ITaS',
        name = name,
        epochs = 25,
        patience = 0,
        batch = 5,
        imgsz = 640,
        seed=seed
    )
    wandb.finish()

In [ ]:
for i in range(5):
    train_yolo_model(main_experiment='least_confidence', name=f'1_train', seed=(i + 1) * 4)

In [ ]:
for i in range(5):
    train_yolo_model(main_experiment='least_confidence', name=f'10_train', seed=(i + 1) * 5)

In [ ]:
for i in range(5):
    train_yolo_model(main_experiment='least_confidence', name=f'20_train', seed=(i + 1) * 6)